# Some experiments on a 5 substations test case

To demonstrate the use of the grid2op framework, we propose here to show how to make some quick studies on a purely fictious test case, a 5 bus system.

This system should not be used for deep research purpose. It is provided here as an example and a tool to get used to how some analyses can be performed using grid2op.

First, we can create a environment:

In [1]:
import grid2op
env = grid2op.make("case5_example")

Now we can see how well the "do nothing" agent (the most basic imaginable) can performe, using some "gym like" methods

In [2]:
from grid2op.Agent import DoNothingAgent
my_agent = DoNothingAgent(env.action_space)

Now do the standard gym loop, and we save all the observations:

In [3]:
all_obs = []
obs = env.reset()
all_obs.append(obs)
reward = env.reward_range[0]
done = False
nb_step = 0
while True:
    action = my_agent.act(obs, reward, done)
    obs, reward, done, _ = env.step(action)
    if done:
        break
    all_obs.append(obs)
    nb_step += 1

We can check if the episode has been completed, or if there has been a game over:

In [4]:
print("Number of timesteps computed: {}".format(nb_step))
print("Total maximum number of timesteps possible: {}".format(env.chronics_handler.max_timestep()))

Number of timesteps computed: 97
Total maximum number of timesteps possible: 2016


As we see here, there is a game over: the agent successfully managed 97 timesteps, while the episode could have lasted 2016.

Let's try to investigate this, for example by plotting the last observation. First we need to create a utilities that will make the plot:

In [5]:
from grid2op.PlotPlotly import PlotObs
# init the plot
graph_layout =  [(0,0), (0,400), (200,400), (400, 400), (400, 0)]
plot_helper = PlotObs(substation_layout=graph_layout,
                      observation_space=env.observation_space)

In [8]:
last_obs = all_obs[-1]
fig = plot_helper.plot_observation(last_obs)
fig.show()

As we can see, the last obsevation is pretty clear: 4 powerlines have been disconnected, thus isolating the load on the bottom right. This lead to a game over. 

It's also possible, of course, to inspect the previous state, just before this one:

In [11]:
previous_act = all_obs[-2]
fig = plot_helper.plot_observation(previous_act)
fig.show()

And now we can know the cause for this problem: all the powerlines that could provide power on the bottom right load are overloaded in this situation, so the protection worked and disconnect them to prevent hurting the surroundings.